In [1]:
import spacy
import random
from spacy.training.example import Example
from pathlib import Path

# --- 1. PREPARAÇÃO DOS DADOS DE TREINO ---
# Defina exemplos de frases para cada intenção.
# O formato é: ("texto da frase", {"cats": {"nome_da_intencao": 1.0, "outra_intencao": 0.0}})
# O 1.0 indica que a frase pertence a essa categoria, e 0.0 que não pertence às outras.

TRAIN_DATA = [
    ("oi", {"cats": {"saudacao": 1.0, "despedida": 0.0, "ver_saldo": 0.0}}),
    ("olá, tudo bem?", {"cats": {"saudacao": 1.0, "despedida": 0.0, "ver_saldo": 0.0}}),
    ("bom dia", {"cats": {"saudacao": 1.0, "despedida": 0.0, "ver_saldo": 0.0}}),
    
    ("tchau", {"cats": {"saudacao": 0.0, "despedida": 1.0, "ver_saldo": 0.0}}),
    ("até logo", {"cats": {"saudacao": 0.0, "despedida": 1.0, "ver_saldo": 0.0}}),
    ("vou indo", {"cats": {"saudacao": 0.0, "despedida": 1.0, "ver_saldo": 0.0}}),

    ("qual é o meu saldo?", {"cats": {"saudacao": 0.0, "despedida": 0.0, "ver_saldo": 1.0}}),
    ("quanto dinheiro eu tenho?", {"cats": {"saudacao": 0.0, "despedida": 0.0, "ver_saldo": 1.0}}),
    ("me mostra o saldo da conta", {"cats": {"saudacao": 0.0, "despedida": 0.0, "ver_saldo": 1.0}}),
]

def treinar_modelo_de_intencoes():
    """Função para treinar e salvar um modelo de classificação de texto (intenções)."""
    
    # --- 2. CRIAÇÃO DO MODELO E PIPELINE ---
    # Começamos com um modelo em português em branco, pois vamos treinar do zero.
    nlp = spacy.blank("pt")

    # Adiciona o componente de classificação de texto ("textcat") ao pipeline. [10]
    if "textcat" not in nlp.pipe_names:
        textcat = nlp.add_pipe("textcat", last=True)
    else:
        textcat = nlp.get_pipe("textcat")

    # Adiciona os "rótulos" (nossas intenções) ao classificador.
    textcat.add_label("saudacao")
    textcat.add_label("despedida")
    textcat.add_label("ver_saldo")
    
    # --- 3. TREINAMENTO DO MODELO ---
    print("Iniciando o treinamento do modelo de intenções...")
    
    # Desabilita outros componentes do pipeline que não são necessários para o treino.
    pipe_exceptions = ["textcat"]
    unaffected_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]
    
    # O treinamento é um loop de otimização.
    with nlp.select_pipes(disable=unaffected_pipes):
        optimizer = nlp.begin_training()
        
        # Itera 10 vezes sobre os dados de treino para o modelo aprender.
        for itn in range(10):
            random.shuffle(TRAIN_DATA)
            losses = {}
            
            # Atualiza o modelo com os exemplos.
            for text, annotations in TRAIN_DATA:
                doc = nlp.make_doc(text)
                example = Example.from_dict(doc, annotations)
                nlp.update([example], sgd=optimizer, drop=0.35, losses=losses)
            
            print(f"Iteração {itn+1}/10 - Perda: {losses['textcat']:.3f}")

    # --- 4. SALVAR O MODELO TREINADO ---
    output_dir = Path("./meu_modelo_de_intencoes")
    if not output_dir.exists():
        output_dir.mkdir()
    nlp.to_disk(output_dir)
    print(f"\n✅ Modelo salvo com sucesso na pasta: '{output_dir.resolve()}'")
    return output_dir

def testar_modelo(modelo_path):
    """Função para carregar e testar o modelo treinado."""
    print("\n--- INICIANDO TESTE DO MODELO ---")
    
    # Carrega o modelo que acabamos de salvar.
    nlp_test = spacy.load(modelo_path)

    # Lista de frases para testar
    frases_teste = [
        "olá",
        "quanto eu tenho na conta?",
        "adeus",
        "bom dia, como vai?",
        "quero ver meu dinheiro"
    ]
    
    for frase in frases_teste:
        doc = nlp_test(frase)
        # O resultado da predição fica em `doc.cats`, um dicionário com as pontuações para cada intenção.
        # Pegamos a intenção com a maior pontuação.
        intencao_prevista = max(doc.cats, key=doc.cats.get)
        pontuacao = doc.cats[intencao_prevista]
        
        print(f"Frase: '{frase}'\n  -> Intenção Prevista: '{intencao_prevista}' (Confiança: {pontuacao:.2f})")


# --- PONTO DE ENTRADA PRINCIPAL ---
if __name__ == "__main__":
    # 1. Treina o modelo e obtém a pasta onde foi salvo
    caminho_modelo = treinar_modelo_de_intencoes()
    
    # 2. Testa o modelo carregando-o da pasta
    testar_modelo(caminho_modelo)

Iniciando o treinamento do modelo de intenções...
Iteração 1/10 - Perda: 2.005
Iteração 2/10 - Perda: 1.945
Iteração 3/10 - Perda: 1.861
Iteração 4/10 - Perda: 1.583
Iteração 5/10 - Perda: 1.439
Iteração 6/10 - Perda: 0.992
Iteração 7/10 - Perda: 0.827
Iteração 8/10 - Perda: 0.379
Iteração 9/10 - Perda: 0.170
Iteração 10/10 - Perda: 0.055

✅ Modelo salvo com sucesso na pasta: 'C:\Users\jonat\Documents\GitHub\NLU\notebooks\meu_modelo_de_intencoes'

--- INICIANDO TESTE DO MODELO ---
Frase: 'olá'
  -> Intenção Prevista: 'saudacao' (Confiança: 0.93)
Frase: 'quanto eu tenho na conta?'
  -> Intenção Prevista: 'ver_saldo' (Confiança: 0.97)
Frase: 'adeus'
  -> Intenção Prevista: 'ver_saldo' (Confiança: 0.66)
Frase: 'bom dia, como vai?'
  -> Intenção Prevista: 'saudacao' (Confiança: 0.84)
Frase: 'quero ver meu dinheiro'
  -> Intenção Prevista: 'ver_saldo' (Confiança: 0.86)


In [10]:
import spacy
import random
from spacy.training.example import Example
from pathlib import Path

# --- 1. PREPARAÇÃO DOS DADOS DE TREINO ---
# Mesmos dados de treino do exemplo anterior. Mais dados melhoram o modelo!
TRAIN_DATA = [
    ("oi", {"cats": {"saudacao": 1.0, "despedida": 0.0, "ver_saldo": 0.0}}),
    ("olá, tudo bem?", {"cats": {"saudacao": 1.0, "despedida": 0.0, "ver_saldo": 0.0}}),
    ("bom dia", {"cats": {"saudacao": 1.0, "despedida": 0.0, "ver_saldo": 0.0}}),
    ("eae", {"cats": {"saudacao": 1.0, "despedida": 0.0, "ver_saldo": 0.0}}),
    
    ("tchau", {"cats": {"saudacao": 0.0, "despedida": 1.0, "ver_saldo": 0.0}}),
    ("até logo", {"cats": {"saudacao": 0.0, "despedida": 1.0, "ver_saldo": 0.0}}),
    ("vou indo", {"cats": {"saudacao": 0.0, "despedida": 1.0, "ver_saldo": 0.0}}),
    ("falou", {"cats": {"saudacao": 0.0, "despedida": 1.0, "ver_saldo": 0.0}}),

    ("qual é o meu saldo?", {"cats": {"saudacao": 0.0, "despedida": 0.0, "ver_saldo": 1.0}}),
    ("quanto dinheiro eu tenho?", {"cats": {"saudacao": 0.0, "despedida": 0.0, "ver_saldo": 1.0}}),
    ("me mostra o saldo da conta", {"cats": {"saudacao": 0.0, "despedida": 0.0, "ver_saldo": 1.0}}),
    ("verificar saldo", {"cats": {"saudacao": 0.0, "despedida": 0.0, "ver_saldo": 1.0}}),
]

# Define o nome da pasta onde o modelo será salvo
MODELO_DIR = Path("../models/meu_modelo_interativo")

def treinar_se_necessario():
    """
    Verifica se o modelo já existe. Se não, treina um novo.
    """
    # if MODELO_DIR.exists():
    #     print(f"Modelo já treinado encontrado em '{MODELO_DIR}'. Carregando...")
    #     return
        
    print("Modelo não encontrado. Iniciando novo treinamento...")
    
    # Cria um pipeline em branco para o português
    nlp = spacy.load("pt_core_news_md")
    
    # Adiciona e configura o classificador de texto (textcat)
    textcat = nlp.add_pipe("textcat", last=True)
    labels = set(label for _, annots in TRAIN_DATA for label in annots["cats"])
    for label in labels:
        textcat.add_label(label)
        
    # Inicia o treinamento
    pipe_exceptions = ["textcat"]
    unaffected_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]
    
    with nlp.select_pipes(disable=unaffected_pipes):
        optimizer = nlp.begin_training()
        for i in range(30):
            random.shuffle(TRAIN_DATA)
            losses = {}
            for text, annotations in TRAIN_DATA:
                doc = nlp.make_doc(text)
                example = Example.from_dict(doc, annotations)
                nlp.update([example], sgd=optimizer, drop=0.35, losses=losses)
            print(f"Iteração {i+1}/10 - Perda: {losses['textcat']:.3f}")

    # Salva o modelo treinado no disco
    nlp.to_disk(MODELO_DIR)
    print(f"\n✅ Modelo treinado e salvo em '{MODELO_DIR.resolve()}'")

def teste_interativo():
    """
    Carrega o modelo treinado e permite que o usuário o teste interativamente.
    """
    print("\n--- TESTE INTERATIVO DE INTENÇÕES ---")
    print('Digite suas frases para testar o modelo. Digite "sair" para terminar.')
    
    try:
        # Carrega o modelo da pasta
        nlp_test = spacy.load(MODELO_DIR)
    except OSError:
        print(f"ERRO: Não foi possível encontrar o modelo em '{MODELO_DIR}'.")
        print("Execute o script uma vez para treinar e salvar o modelo primeiro.")
        return
        
    while True:
        # Pede uma frase ao usuário
        frase_usuario = input("\nSua frase: ")
        
        if frase_usuario.lower() == "sair":
            print("Até logo!")
            break
            
        # Processa a frase com o modelo carregado
        doc = nlp_test(frase_usuario)
        
        # Pega a intenção com a maior pontuação do dicionário doc.cats
        # O dicionário contém as pontuações de confiança para cada rótulo.
        if doc.cats:
            intencao_prevista = max(doc.cats, key=doc.cats.get)
            pontuacao = doc.cats[intencao_prevista]
            
            print(f"  -> Intenção Prevista: '{intencao_prevista}' (Confiança: {pontuacao:.2f})")
        else:
            print("Não foi possível prever a intenção.")

# --- PONTO DE ENTRADA PRINCIPAL ---
if __name__ == "__main__":
    # 1. Garante que o modelo esteja treinado
    treinar_se_necessario()
    
    # 2. Inicia o modo de teste interativo
    teste_interativo()

Modelo não encontrado. Iniciando novo treinamento...
Iteração 1/10 - Perda: 2.675
Iteração 2/10 - Perda: 2.553
Iteração 3/10 - Perda: 2.253
Iteração 4/10 - Perda: 1.750
Iteração 5/10 - Perda: 1.505
Iteração 6/10 - Perda: 0.831
Iteração 7/10 - Perda: 0.737
Iteração 8/10 - Perda: 0.149
Iteração 9/10 - Perda: 0.210
Iteração 10/10 - Perda: 0.061
Iteração 11/10 - Perda: 0.006
Iteração 12/10 - Perda: 0.005
Iteração 13/10 - Perda: 0.002
Iteração 14/10 - Perda: 0.002
Iteração 15/10 - Perda: 0.002
Iteração 16/10 - Perda: 0.001
Iteração 17/10 - Perda: 0.000
Iteração 18/10 - Perda: 0.000
Iteração 19/10 - Perda: 0.000
Iteração 20/10 - Perda: 0.000
Iteração 21/10 - Perda: 0.000
Iteração 22/10 - Perda: 0.000
Iteração 23/10 - Perda: 0.000
Iteração 24/10 - Perda: 0.000
Iteração 25/10 - Perda: 0.000
Iteração 26/10 - Perda: 0.000
Iteração 27/10 - Perda: 0.000
Iteração 28/10 - Perda: 0.000
Iteração 29/10 - Perda: 0.000
Iteração 30/10 - Perda: 0.000

✅ Modelo treinado e salvo em 'C:\Users\jonat\Documents\